## `Занятие 1.2: Основы Pytorch`

#### `Сириус, смена "Алгоритмы и анализ данных" 2024`

О чём можно узнать из этого ноутбука:

* операции при работе с массивами
* многомерные массивы
* изменение размеров массивов
* broadcasting
* продвинутая индексация
* view и копирование
* свёртка
* разные прикладные задачи

In [1]:
import warnings
warnings.filterwarnings('ignore')

### `Представление матрицы в Python`

Простейший вариант - список списков:

In [2]:
A = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9],
]

B = [
    [1, 0, 0],
    [0, 2, 0],
    [0, 0, 3],
]

def print_matrix(A):
    for row in A:
        print(row)
    print()

print_matrix(A)
print_matrix(B)

[1, 2, 3]
[4, 5, 6]
[7, 8, 9]

[1, 0, 0]
[0, 2, 0]
[0, 0, 3]



### `Представление матрицы в Python`

Одна из проблем списков — отсутствие поэлементных и матричных операций.

Другой недостаток — работа со списками не позволяет использовать векторные инструкции в процессоре, которые на порядки ускоряют матричные вычисления

Попробуем самостоятельно реализовать несколько методов для матричных вычислений, работающих с представлением матриц в виде списков.

Для оценки производительности будем использовать *декооратор* `timed`:

In [4]:
from time import perf_counter

def timed(method):
    def __timed(*args, **kw):
        time_start = perf_cointer()
        result = method(*args, **kw)
        time_end = perf_cointer()
        print('{}  {:.3f} ms\n'.format(method.__name__,
                                      (time_end - time_start) * 1000))
        return result

    return __timed

### `Опишем базовые операции: транспонирование`

Для матрицы $A\in\text{Mat}(m\times n)$:
$$
C_{ij} := A_{ji}
$$

In [ ]:
@timed
def matrix_transpose(A):
    # your code here
    ...

print_matrix(matrix_transpose(A))

### `Опишем базовые операции: сложение`

Для матриц одинакового размера $A,B\in\text{Mat}(m\times n)$:

$$
C_{ij}:=A_{ij}+B_{ij}
$$

In [ ]:
@timed
def matrix_add(A, B):
    # skip correctness of dimensions check

    n, m = len(A), len(A[0])
    C = [[0.0 for _ in range(m)] for _ in range(n)]

    for i in range(n):
        for j in range(m):
            C[i][j] = A[i][j] + B[i][j]
    return C

print_matrix(matrix_add(A, B))

matrix_add  0.013 ms

[2, 2, 3]
[4, 7, 6]
[7, 8, 12]



### `Опишем базовые операции: умножение`

Для матрицы $A\in\text{Mat}(m\times n)$, $B\in\text{Mat}(n\times m)$:
$$
C_{ij}=\sum_{r=1}^n A_{ir}B_{rj}
$$

Анимированный пример:

![](sem_1_assets/matrix-multiplication.gif)

In [9]:
def dot(a, b):
    return sum(x * y for (x, y) in zip(a, b))

@timed
def matrix_mul(A, B):
    # check correctness of dimensions
    n, m = len(A), len(B[0])
    C = [[0.0 for _ in range(n)] for _ in range(m)]

    B_T = matrix_transpose(B)
    for i in range(n):
        for j in range(m):
            C[i][j] = dot(A[i], B_T[j])
    return C

print_matrix(matrix_mul(A, B))

matrix_transpose  0.009 ms

matrix_mul  1.176 ms

[1, 4, 9]
[4, 10, 18]
[7, 16, 27]



### `Библиотека PyTorch`

- Позволяет работать с многомерными массивами (**тензорами**)
- Реализует множество базовых матричных операций
- Работает на порядки быстрее самописных реализаций в Python

In [ ]:
import torch

### `Почему torch быстрый`

- Значительная часть кода написана на C++ и CUDA

- Базовым классом является `Tensor`, имеющий следующие отличия от списков:
    1. Имеет фиксированную длину, задаваемую в момент его создания (списки в Python могут менять размер динамически)
    2. Все элементы в NumPy array имеют один тип
- PyTorch поддерживает ускорение с помощью GPU, что позволяет использовать возможности параллельной обработки. Это может привести к ускорению обучения и использования моделей глубокого обучения на порядок
- И миллион других причин


### `Способы создания torch.Tensor`

1. Из списка списков
2. Пустой
3. Заполненный нулями
4. Заполненный единицами
5. Заполненный нужным значением

In [ ]:
print(torch.tensor([[1, 2, 3], [4, 5, 6]]))

In [ ]:
print(torch.empty(shape=[2, 3]))  # values are arbitrary

[[4.89521082e-310 0.00000000e+000 2.05833592e-312]
 [6.79038654e-313 2.14321575e-312 2.27053550e-312]]


In [ ]:
print(torch.zeros([2, 3]))

[[0. 0. 0.]
 [0. 0. 0.]]


In [ ]:
print(torch.ones([2, 3]))

[[1. 1. 1.]
 [1. 1. 1.]]


In [ ]:
print(torch.full([2, 3], 3.0))

[[3. 3. 3.]
 [3. 3. 3.]]


Некорректные входные данные приведут к ошибке:

In [ ]:
torch.tensor([[1, 2, 3], [4, 6]])

ValueError: ignored

In [ ]:
# Синтаксис аналогичен функции `range`
torch.arange(2, 10)

array([2, 3, 4, 5, 6, 7, 8, 9])

### `Способы создания torch.Tensor`

<font color='brown'>**Задача 2.** Задайте массив `result` размером $50$ на $30$, состоящий из троек:</font>

In [156]:
### your code here

<font color='brown'>**Задача 3.** Какая размерность будет у массива, полученного с помощью команды `np.array([[1], [2]])`?</font>

In [155]:
### your answer here

### `Арифметические операции`

- Арифметические операции в общем случае по-элементные и требуют одинакового размера операндов<br>

- Но часто NumPy может применять их к операндам разного размера с помощью broadcasting, то есть правил обработки операндов разного размера<br><br>

Примеры операций с массивами одного размера:

In [ ]:
A, B = torch.array(A), torch.array(B)  # were declared previously

print(A, '\n\n', B, '\n\n', 2 * A, '\n\n', A + B)

[[1 2 3]
 [4 5 6]
 [7 8 9]] 

 [[1 0 0]
 [0 2 0]
 [0 0 3]] 

 [[ 2  4  6]
 [ 8 10 12]
 [14 16 18]] 

 [[ 2  2  3]
 [ 4  7  6]
 [ 7  8 12]]


### `Поэлементные арифметические операции`

In [ ]:
torch.sin(A)

array([[ 0.84147098,  0.90929743,  0.14112001],
       [-0.7568025 , -0.95892427, -0.2794155 ],
       [ 0.6569866 ,  0.98935825,  0.41211849]])

In [ ]:
torch.abs(A)

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [32]:
A ** 0.5

array([[1.        , 1.41421356, 1.73205081],
       [2.        , 2.23606798, 2.44948974],
       [2.64575131, 2.82842712, 3.        ]])

Поэлементые операции сравнения:

In [33]:
A > 3

array([[False, False, False],
       [ True,  True,  True],
       [ True,  True,  True]])

### `Матричные арифметические операции`

In [34]:
print(A - B)

[[0 2 3]
 [4 3 6]
 [7 8 6]]


In [35]:
print(A * B)

[[ 1  0  0]
 [ 0 10  0]
 [ 0  0 27]]


In [36]:
print(B / A)

[[1.         0.         0.        ]
 [0.         0.4        0.        ]
 [0.         0.         0.33333333]]


In [37]:
print(A @ B)

[[ 1  4  9]
 [ 4 10 18]
 [ 7 16 27]]


### `Матричное умножение`

Расмотрим для случая двумерных матриц

In [ ]:
print(A @ B, '\n')
print(A.dot(B), '\n')
print(torch.dot(A, B))

[[ 1  4  9]
 [ 4 10 18]
 [ 7 16 27]] 

[[ 1  4  9]
 [ 4 10 18]
 [ 7 16 27]] 

[[ 1  4  9]
 [ 4 10 18]
 [ 7 16 27]]


### `Транспонирование`

- Расмотрим для случая двумерных матриц, кому интересны многомерные, изучайте
https://stackoverflow.com/questions/32034237/how-does-numpys-transpose-method-permute-the-axes-of-an-array
- При транспонировании (как и `reshape`) возвращается ссылка на те же данные

In [ ]:
print(A, '\n\n', A.T, '\n\n', A.transpose())

[[1 2 3]
 [4 5 6]
 [7 8 9]] 

 [[1 4 7]
 [2 5 8]
 [3 6 9]] 

 [[1 4 7]
 [2 5 8]
 [3 6 9]]


### `Сравнение скорости`

Воспользуемся декоратором `timed` для сравнения скорости матричных операций, реализованных в `numpy` и написанных выше с помощью списков:

In [62]:
@timed
def matrix_add_np(A, B): return A + B

@timed
def matrix_mul_np(A, B): return A @ B

In [67]:
tmp = [range(1000) for _ in range(1000)]
X, Y = np.array(tmp), np.array(tmp)

_ = matrix_add(X, Y)
_ = matrix_add_np(X, Y)

tmp = [range(200) for _ in range(200)]
X, Y = np.array(tmp), np.array(tmp)

_ = matrix_mul(X, Y)
_ = matrix_mul_np(X, Y)

matrix_add  690.307 ms

matrix_add_np  3.761 ms

matrix_transpose  12.119 ms

matrix_mul  2138.916 ms

matrix_mul_np  8.289 ms



## `Спасибо за внимание!`